In [1]:
import wbdata
import pandas as pd
import tqdm
import numpy as np

from scipy.stats import norm, uniform, gaussian_kde, multivariate_normal
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from numba import njit
from see import see

In [2]:
#----------------------------------------------------------------------------------------------------------------------------------------------------------------
# Components
#----------------------------------------------------------------------------------------------------------------------------------------------------------------
import json
def txt_load(filename):
    l = []
    with open(filename, "r") as f:
        for line in f:
            l.append(line.rstrip())
    return l

def txt_dump(filename, item):
    with open(filename, "w") as f:
        for line in item:
            f.write(line + "\n")
    print("item saved to txt file \'"+ str(filename) +"\'")

def json_load(filename):
    with open(filename, "r") as f:
        j = json.load(f)
    return j

def json_dump(filename, item):
    with open(filename, "w") as f:
        json.dump(item, f)
    print("item saved to json file \'"+ str(filename) +"\'")

In [3]:
#########################################################################################################################################################################
#　世界銀行APIからデータを取得
#########################################################################################################################################################################
def create_column(wbd,osh,target):
    l = []
    codes = json_load("country-codes.json")
    for i in range(len(osh)):
        code = osh.iloc[i]["code"]
        year = str(osh.iloc[i]["year"])
        c = codes[code]
        try:
            l.append(wbd.loc[c].loc[year][target])
        except:
            l.append("")
    return pd.DataFrame(l)

def get_df(osh, countries, indicators):
    wbd =  wbdata.get_dataframe(indicators, country=countries)
    targets = list(indicators.values())
    for item in targets:
        print("Creating column "+item)
        osh[item] = create_column(wbd,osh,item)
    return(osh)

In [165]:
codes = json_load("country-codes.json")
osh = pd.read_csv("osmarketshare_final_5.csv")
country = codes.keys()

##############################################################################
# 取りたいデータのindicatorをここに入れる
##############################################################################
indicator = {
"NY.GDP.MKTP.CD":"gdp",
"SP.POP.TOTL":"population",
"AG.LND.TOTL.K2":"LandArea",
"SP.POP.0014.TO.ZS":"ChildPercentage", #percentage of population 0-14 years old
"SE.XPD.TOTL.GB.ZS":"GovEduc", #政府支出における教育の割合

"SP.POP.1564.TO.ZS":"WorkingPercentage", #percentage of population 15-64 years old
"SE.COM.DURS":"EduDuration", #義務教育年数

#"SP.DYN.TFRT.IN":"Fertility rate",
#"SP.URB.TOTL.IN.ZS":"UrbanPopulation",
#"IP.JRN.ARTC.SC":"articles",
#"SL.TLF.TOTL.IN":"labor-force",
#"TX.VAL.OTHR.ZS.WT":"computer", #コンピューター・通信・その他サービス(サービス輸出比) - Computer, communications and other services (% of commercial service exports)
}

In [ ]:
#　世界銀行から任意のデータを取得
df = get_df(osh,country,indicator)
df

In [167]:
# 取ったデータをCSVで保存

df.to_csv("wb_1212.csv")

In [ ]:
# CSV読み込み

df = pd.read_csv("wb_1210.csv")
df

In [ ]:
# df加工（デバイスのシェア結合）

desktop = pd.read_csv("./osmarketshare_total-desktop.csv")
desktop["year"] = [ x.split("-")[0] for x in desktop["Date"] ]
desktop["sortname"] = desktop["Country"] + desktop["year"]
desktop = desktop.groupby(by="sortname").mean()
desktop["year"] = [ int(x[-4:]) for x in desktop.index ]

In [ ]:
#　グラフ化
x="EduDuration"
y="iOS"
df[df["year"]==2020].plot.scatter(x=x, y=y)

In [849]:
##########################################################################################################################################################################################
# クロスセクション
###########################################################################################################################################################################################


df = pd.read_csv("wb_1212.csv")
df = df.sort_values(["country","year"])

# 一人当たりGDP・人口密度対数化
df["percapita"] = df["gdp"] / df["population"]
df["percapita_log"] = np.log10(df["gdp"] / df["population"] +1)
df["population_log"] = np.log10(df["population"] +1)
df["density"] = np.log10(df["LandArea"] / df["population"]+1)
df["ios"] = df["iOS"]/(df["Android"]+df["iOS"])



#Macシェア率
desktop = pd.read_csv("./osmarketshare_total-desktop.csv")
desktop["year"] = [ x.split("-")[0] for x in desktop["Date"] ]
desktop["sortname"] = desktop["Country"] + desktop["year"]
desktop = desktop.groupby(by="sortname").mean()
desktop["year"] = [ int(x[-4:]) for x in desktop.index ]

osxl = []
for x in df["sortname"]:
    osx = desktop.loc[x]["OS X"]
    osxl.append(osx)

#言語・民族・宗教ダミー
dummies = pd.read_csv("dummies.csv")
dummies["NonReli"] = dummies["NonReli"].fillna(0)
dummies = dummies.drop(columns=["Unnamed: 0","BritanicaLink","Lang"])
ds = pd.DataFrame()
for item in df["country"].tolist():
    ds = pd.concat([ds, dummies[dummies["cs"]==item]])



df = pd.concat([df,ds.reset_index(), pd.DataFrame(osxl, columns=["OSX"])],axis=1)

df = df.drop(columns=["Unnamed: 0.3", "Unnamed: 0.2","Unnamed: 0.1","Unnamed: 0"])

In [879]:
#----------------------------------------------------------------------------------------------------------------------------------------------------------------
# 年を決める
#----------------------------------------------------------------------------------------------------------------------------------------------------------------

data = df[df["year"]==2020]
data.to_csv("df_1214.csv")

In [ ]:
# 最終的に使用したデータ

data = pd.read_csv("df_1214_2.csv")
tbl = data[["iOS","percapita_log","LangCount","EthnicCount","ReliCount","NonReli"]].describe()
tbl = tbl.transpose()
tbl

In [1103]:
l = ["Iraq","Kenya","Sweden","Denmark","Nigeria","Zimbabwe","Sri Lanka"]
df = pd.DataFrame()
for x in l:
    df = pd.concat([
        df,
        data[data["country"]==x]
    ])
df.to_csv("aaa.csv")

In [ ]:
#formula = "iOS ~ gdp + population + LandArea + UrbanPopulation + ChildPercentage + WorkingPercentage"
#data = data.dropna()
l = [
"percapita_log",
"island",
"LangCount",
"EthnicCount",
"ReliCount",
"NonReli",
"store"
]
formula = "iOS ~ "
for item in l:
    
    formula += item + " + "
formula = formula[0:-2]
l.append("iOS")
res = ols(formula, data=data[l].dropna()).fit()
res.summary()

In [1092]:
#----------------------------------------------------------------------------------------------------------------------------------------------------------------
# パネル 月毎　VAR
#----------------------------------------------------------------------------------------------------------------------------------------------------------------
codes = json_load("country-codes.json")

m = pd.read_csv("./osmarketshare_total.csv")
d = pd.read_csv("./osmarketshare_total-desktop.csv")

mobile = pd.DataFrame()
desktop = pd.DataFrame()


for value in list(codes.values()):
    mobile = pd.concat([
        mobile,
        m[m["Country"]==value]
    ])
    desktop = pd.concat([
        desktop,
        d[d["Country"]==value]
    ])

desktop = desktop[desktop["Date"] != "2022-11"]
mobile = mobile.reset_index().drop(columns = ["index", "Unnamed: 0"])
desktop = desktop.reset_index().drop(columns = ["index", "Unnamed: 0"]).rename(columns = {"OS X": "OSX"})

dm = {
    "iOS": "iOS_lag0",
}
dd = {
    "OSX": "OSX_lag0"
}

panel = pd.concat([
    mobile,
    mobile.groupby("Country")[["iOS"]].diff().rename(columns = dm)
],axis=1)

panel = pd.concat([
    panel,
    desktop.groupby("Country")[["OSX"]].diff().rename(columns = dd)
],axis=1)




In [ ]:
codes = json_load("country-codes.json")
pd.options.display.float_format = '{:.4f}'.format
cdf = pd.DataFrame()
ex_cl = []
res = pd.DataFrame()
for c in list(codes.values()):
    df = panel[panel["Country"]==c][["iOS_lag0","OSX_lag0"]].dropna()
    df = df.reset_index().drop(columns=["index"])
    # 最大のラグ数
    maxlags = 26
    # モデルのインスタンス生成
    var_model = VAR(df)
    # 最適なラグの探索
    lag = var_model.select_order(maxlags).selected_orders
    # モデルの学習
    if lag["aic"] != 0:
        results = var_model.fit(lag["aic"])
        test_results = results.test_causality(causing=0, caused=1)  
        test_results1 = results.test_causality(causing=1, caused=0)  
        tf = False
        if test_results.pvalue<0.05:
            test_results = results.test_causality(causing=0, caused=1)  
            test_results1 = results.test_causality(causing=1, caused=0)  
            results = var_model.fit(lag["aic"])
            index_list = list(results.params.index)
            ll = "L"+ str(lag["aic"]) + "." + test_results.causing[0]
            dd = pd.DataFrame([
            [
                c,
                lag["aic"],
                test_results.causing[0].split("_")[0] + " -> " + test_results.caused[0].split("_")[0],
                results.params.loc[ll][test_results.causing[0]],
                results.tvalues.loc[ll][test_results.causing[0]],
                test_results.pvalue.round()
            ]
            ],columns=["country","lag","vector","param","t","p"])
            cdf = pd.concat([cdf,dd])

        if test_results1.pvalue<0.05:
            ll = "L"+ str(lag["aic"]) + "." + test_results1.causing[0]
            dd = pd.DataFrame([
            [
                c,
                lag["aic"],
                test_results1.causing[0].split("_")[0] + " -> " + test_results1.caused[0].split("_")[0],
                results.params.loc[ll][test_results1.caused[0]],
                results.tvalues.loc[ll][test_results1.caused[0]],
                test_results1.pvalue
            ]
            ],columns=["country","lag","vector","param","t","p"])
            cdf = pd.concat([cdf,dd])


In [ ]:
cdf[cdf["lag"]<26].sort_values(["lag","country"])

In [1087]:
cdf[cdf["lag"]<28].sort_values(["lag","country"]).to_csv("1214_ex_28.csv")

In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------------------------------------
# （パネル 年ごと）　　
#----------------------------------------------------------------------------------------------------------------------------------------------------------------

names={
    "iOS":"iOS_lag0",
    "OSX":"OSX_lag0",
    "percapita_log":"percapita_log_lag0",
    "WorkingPercentage":"WorkingPercentage_lag0",
}

panel = pd.concat([
        df[["country","year","sortname","iOS","percapita","percapita_log","WorkingPercentage","OSX"]],
        #df.groupby("country")[["iOS", "OSX", "percapita_log", "WorkingPercentage"]].diff().rename(columns = names)
        df.groupby("country")[["iOS", "OSX", "percapita_log", "WorkingPercentage"]].diff().rename(columns = names)
    ],axis=1)

def take_lag(df,dic,lag):
    return df.groupby("country")[list(dic.keys())].shift(lag).rename(columns = dic)

for i in range(1,11):
    names = {
        "iOS_lag%s"%(i-1) : "iOS_lag%s"%i,
        "OSX_lag%s"%(i-1): "OSX_lag%s"%i,
        "WorkingPercentage_lag%s"%(i-1): "WorkingPercentage_lag%s"%i,
        "percapita_log_lag%s"%(i-1): "percapita_log_lag%s"%i
    }
    panel = pd.concat([
        panel,
        take_lag(panel, names, i)
    ],axis=1)

panel

In [ ]:
#formula = "iOS_diff ~ iOS_lag1 + OSX_lag1 "
res = ols(formula, data=panel).fit()
print(res.summary())